In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
!pwd

In [3]:
!pwd

/home/nasonn/code/AltInt/Inflation_prediction/notebooks


In [5]:
df_rpi_cpi = pd.read_csv("RPI_CPI.csv")


In [ ]:
df_rpi_cpi['Date'] = pd.to_datetime(df_rpi_cpi['Date'], infer_datetime_format=True)

In [ ]:
df = pd.read_csv('BloombergData-01a.csv')
df

In [ ]:
df = df.join(df_rpi_cpi)
df

In [ ]:
##Converting dates to date- time objects
columns_list = list(df.columns)
for columns in columns_list:
    if 'Date' in columns:
        df[columns] = pd.to_datetime(df[columns], infer_datetime_format=True)

In [ ]:
df.info()

In [ ]:
CO1_Comdty = df.groupby(pd.PeriodIndex(df['Date CO1 Comdty'], freq='M'))[['CO1 Comdty']].mean()
CL1_Comdty = df.groupby(pd.PeriodIndex(df['Date CL1 Comdty'], freq='M'))[['CL1 Comdty']].mean()
GBP_Curncy = df.groupby(pd.PeriodIndex(df['Date GBP Curncy'], freq='M'))[['GBP Curncy']].mean()
GBPEUR_Curncy = df.groupby(pd.PeriodIndex(df['Date GBPEUR Curncy'], freq='M'))[['GBPEUR Curncy']].mean()
UKX_Index = df.groupby(pd.PeriodIndex(df['Date UKX Index'], freq='M'))[['UKX Index']].mean()
SPX_Index = df.groupby(pd.PeriodIndex(df['Date SPX Index'], freq='M'))[['SPX Index']].mean()          
SX5E_Index = df.groupby(pd.PeriodIndex(df['Date SX5E Index'], freq='M'))[['SX5E Index']].mean()
BPSW2_Curncy = df.groupby(pd.PeriodIndex(df['Date BPSW2 Curncy'], freq='M'))[['BPSW2 Curncy']].mean()
BPSW10_Curncy = df.groupby(pd.PeriodIndex(df['Date BPSW10 Curncy'], freq='M'))[['BPSW10 Curncy']].mean()
TTFG1MON_Index = df.groupby(pd.PeriodIndex(df['Date TTFG1MON Index'], freq='M'))[['TTFG1MON Index']].mean()
RPI = df.groupby(pd.PeriodIndex(df['Date'], freq='M'))[['RPI']].mean()
CPI = df.groupby(pd.PeriodIndex(df['Date'], freq='M'))[['CPI']].mean()

In [ ]:
df_list = [CO1_Comdty,CL1_Comdty,GBP_Curncy,GBPEUR_Curncy,UKX_Index,SPX_Index,SX5E_Index,TTFG1MON_Index, RPI, CPI]

In [ ]:
### MAKING A HUGE DATASET BY JOINING ALL COLUMNS WITH KEY INDEX BEING DATE
result_df = df_list[0]
for i in range(len(df_list)-1):
    if i<len(df_list):
        result_df = result_df.join(df_list[i+1], how='inner')

In [ ]:
##### NOTE WE ARE ONLY CONSIDERING DATA AFTER 2008. HENCE: 2009

result_df = result_df[result_df.index>="2009-01"]

In [ ]:
result_df

In [ ]:
#finding the correlation between the external features and RPI/CPI index.This will help to see which external features are
# best to use in time series model
corr_matrix = result_df.corr()
sns.set_style("dark")
sns.heatmap(corr_matrix, annot=True,cmap = 'PiYG', vmin=-1.0, vmax = 1.0)

In [ ]:
### Converting result_df index into datetime64 for ease of use

result_df.index = result_df.index.to_timestamp()

In [ ]:
result_df.index

In [ ]:
#### Converting the index values in GBP to standard units

result_GBP_df = result_df.copy()

result_GBP_df['CO1 Comdty'] = result_df['CO1 Comdty']/result_df['GBP Curncy']
result_GBP_df['CL1 Comdty'] = result_df['CL1 Comdty']/result_df['GBP Curncy']
result_GBP_df['TTFG1MON Index'] = result_df['TTFG1MON Index']/result_df['GBP Curncy']
result_GBP_df['SPX Index'] = result_df['SPX Index']/result_df['GBP Curncy']

result_GBP_df['SX5E Index'] = result_df['SX5E Index']/result_df['GBPEUR Curncy']

result_GBP_df

# XGBOOST

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error
plt.style.use('seaborn-whitegrid')

In [ ]:
### We are only considering time after the stock market crash. Hence, the "split_date"

cutoff_date = "2009-01-01"
XGB_df = result_GBP_df.loc[result_GBP_df.index >= cutoff_date].copy()

In [ ]:
XGB_df

In [ ]:
_ = XGB_df.plot( figsize=(15,5), title='Time-series data of all index')
plt.legend(loc='best')
plt.yscale('log')

## Train test split

In [ ]:
XGB_train = XGB_df.iloc[:114]
XGB_test = XGB_df.iloc[114:]

In [ ]:
plt.plot(XGB_train)
plt.plot(XGB_test)
plt.yscale('log')

In [ ]:
XGB_df['TTFG1MON Index'].plot()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

TTFG_decomp_train = seasonal_decompose(XGB_train['TTFG1MON Index'],model = 'multiplicative')
TTFG_decomp_test = seasonal_decompose(XGB_test['TTFG1MON Index'],model = 'multiplicative')

In [ ]:
TTFG_decomp_test.seasonal.loc["2019-01-01":"2022-01-01"].shape

In [ ]:
from statsmodels.tsa.stattools import adfuller

adfuller(TTFG_decomp_train.resid.dropna())

In [ ]:
# TTFG_decomp_train.resid.dropna(inplace = True)
# TTFG_decomp_test.resid.dropna(inplace = True)

In [ ]:
### Credits to Rob Mulla from Kaggle: https://www.kaggle.com/code/robikscube/tutorial-time-series-forecasting-with-xgboost
def create_features(df, label=None):
    """
    Creates time series features from datetime index
    """
    df['date'] = df.index
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    
    X = df[['hour','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear']]
    if label:
        y = df[label]
        return X, y
    return X

In [ ]:
X_train,y_train= create_features(pd.DataFrame(TTFG_decomp_train.resid), label='resid')
X_test, y_test = create_features(pd.DataFrame(TTFG_decomp_test.resid), label='resid')

In [ ]:
X_train

In [ ]:
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

In [ ]:
reg = xgb.XGBRegressor(n_estimators=1000)
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        early_stopping_rounds=50,
       verbose=False)

In [ ]:
### Feature importance rank
_ = plot_importance(reg, height=0.9)

In [ ]:
### Creating new Dframe for comparison of predicted values and actual values. (Both are decomposed)
d = pd.DataFrame({'TTFG1MON Index After S.decomp': \
                  y_test\
                  *TTFG_decomp_test.seasonal.loc["2019-01-01":"2022-01-01"] \
                  *TTFG_decomp_test.trend.loc["2019-01-01":"2022-01-01"],\
                  'TTFG1MON_Prediction': \
                  reg.predict(X_test) \
                  *TTFG_decomp_test.seasonal.loc["2019-01-01":"2022-01-01"] \
                  *TTFG_decomp_test.trend.loc["2019-01-01":"2022-01-01"]})

In [ ]:
### Simple plot of 3 values 
# TTG1MON Index decomposed and recomposed
# TTG1MON Predicted values from XGB_train
# TTG1MON Index from the ORIGINAL DATASET

d.plot(figsize=(13,7))
XGB_test['TTFG1MON Index'].plot()
plt.legend()
plt.title("Plot of XGB model vs actual data for TTFG1MON Index")
plt.xlabel("Year")
plt.ylabel("Values")

In [ ]:
### Proof trend*seasonal*resid is correct!
XGB_test['TTFG1MON Index'].plot(figsize = (13,7))
d['TTFG1MON Index After S.decomp'].plot()
plt.legend()
plt.xlabel("Year")
plt.ylabel("Values")

## Looking closely at Predicted vs Actual

In [ ]:
# Plotting
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
_ = d[['TTFG1MON Index After S.decomp','TTFG1MON_Prediction']].plot(ax=ax,
                                              style=['-','.'])
ax.set_xbound(lower='01-01-2019', upper='04-01-2020')
plot = plt.suptitle('01-01-2019 to 04-01-2020 Forecast vs Actuals')

## Metrics for XGB_test

In [ ]:
mean_squared_error(y_true=d['TTFG1MON Index After S.decomp'],
                   y_pred=d['TTFG1MON_Prediction'])

In [ ]:
print('p-value: ', adfuller(y)[1])

In [ ]:
print('p-value: ', adfuller(y.diff(1).dropna())[1])

In [ ]:
print('p-value: ', adfuller(y.diff(1).diff(1).fillna(0))[1])

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Original Series
fig, axes = plt.subplots(3, 2, figsize=(13,10))
axes[0, 0].plot(y); axes[0, 0].set_title('Original Series')
plot_acf(y, auto_ylims=True,ax=axes[0, 1])

# 1st Differencing
axes[1, 0].plot(y.diff()); axes[1, 0].set_title('1st Order Differencing')
plot_acf(y.diff().dropna(), auto_ylims=True,ax=axes[1, 1])

# 2nd Differencing
axes[2, 0].plot(y.diff().diff()); axes[2, 0].set_title('2nd Order Differencing')
plot_acf(y.diff().diff().dropna(), auto_ylims=True, ax=axes[2, 1])

plt.tight_layout()

In [ ]:
differenced_rpi = y.diff(12).diff(12).dropna()
RPI_DIF = pd.DataFrame(differenced_rpi)

differenced_SPX = result_df['SPX Index'].diff().dropna()
SPX_DIF  = pd.DataFrame(differenced_SPX)

differenced_df = RPI_DIF.join(SPX_DIF)




In [ ]:
differenced_df['RPI_dif'] = differenced_df['RPI']

In [ ]:
differenced_df['RPI'] = result_df['RPI']

In [ ]:
differenced_df

In [ ]:
result_df.columns

In [ ]:
def adf_and_diff(tseries):
    """
    Takes in a series/dataframe with column specified and determines whether the time series needs to be stationary or not.
    The function attempts to convert it to a stationary time-series by differencing.
    It also tries to the minimize of orders for differencing. (Maximum 2nd-order in works).
    """
    from statsmodels.tsa.stattools import adfuller
    
    p_value = adfuller(tseries)[1]
    
    differenced_series = tseries
    
      
    while p_value >= 0.05:
        
        p_value = adfuller(differenced_series.diff(12).fillna(0))[1]
        
        differenced_series = differenced_series.diff(12).fillna(0)
        
        print(f'p value is reduced to {p_value}')
        
        print("Differenced")
        
        
            
            
            
    return differenced_series

    

In [ ]:
def decomp(tseries):
    """
    Takes in a series/dataframe performs seasonal decomposition on it
    
    """

In [ ]:
adf_and_diff(result_df['RPI'])

In [ ]:
result.resid.plot()

In [ ]:
result_df['RPI']

In [ ]:
result_df['RPI']

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(result_df['RPI'].dropna(0), model='multiplicable', period=12)

In [ ]:
result_df['RPI'].iloc[6:421]

In [ ]:
result_df['RPI'].reset_index()

In [ ]:
recomposed = result.resid.dropna()* result.trend.dropna()*result.seasonal.dropna().iloc[6:421]


In [ ]:
result.seasonal*

In [ ]:
result_df['RPI'].plot()
recomposed.plot()

In [ ]:
result.seasonal.dropna().iloc[6:421]

In [ ]:
adfuller(result.resid.dropna(0))[1]

In [ ]:
result_df.columns 

In [ ]:
adf_and_diff(result_df['GBPEUR Curncy'])

In [ ]:
adfuller(result_df['RPI'].diff(12).diff(12).fillna(0))[1]

In [ ]:
adfuller(result_df['RPI'].diff(12).diff(12).fillna(0))[1]

In [ ]:
result_df['RPI'].diff(12).diff(12).diff(12).dropna()

# NHITS

In [ ]:
from darts import TimeSeries


series1 = TimeSeries.from_dataframe(differenced_df.iloc[4:-12], value_cols = 'RPI_dif',\
                                   fill_missing_dates=True, freq=None)

series2 = TimeSeries.from_dataframe(differenced_df.iloc[4:-12], value_cols = 'SPX Index',\
                                   fill_missing_dates=True, freq=None)



In [ ]:
static_covs_single = pd.DataFrame(data={"cont": [1]})

s1sc = series1.with_static_covariates(static_covs_single)

s2sc = series2.with_static_covariates(static_covs_single)


In [ ]:
from darts.models.forecasting.nhits import NHiTSModel as nhits


model = nhits(input_chunk_length= 100, output_chunk_length = 1,num_stacks = 10, num_blocks = 1)

model.fit([s1sc,s2sc], epochs=200, verbose=True)


In [ ]:
#static_covs_past = pd.DataFrame(data={"cont": [1], "cat": [3]})
pred_ = model.predict(series=s1sc, n=14)

In [ ]:
pred_.plot()
plt.plot(differenced_rpi[4:-12])


In [ ]:
## "Undifferencing" the time series.

undiff_pred = pred_.pd_dataframe().cumsum().apply(lambda x: x+298.4)

In [ ]:
plt.plot(differenced_df[4:-12]['RPI'])
plt.plot(undiff_pred)